<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data_last_changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
Data_basic = pd.DataFrame()

In [5]:
Data_basic = pd.DataFrame(columns=[
    'GameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2014,2023))

In [6]:
seasons

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [7]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [8]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [9]:
Data_basic.to_csv('/content/drive/MyDrive/play_ID_data_2014-1022.csv', index=False)


In [10]:
print(Data_basic[Data_basic['Winner']=='NaN'])





         GameID  season  types  week HomeTeam_ID AwayTeam_ID Winner
174   400554331    2014      2    12           2          20    NaN
782   400951581    2017      2     1          15          27    NaN
2321  401437947    2022      2    17           4           2    NaN


In [11]:
stats_df = pd.DataFrame()
stats_df = pd.DataFrame(columns=['GameID','clock','Pass_Incompletion_Home',
  'Punt_Home',
  'Timeout_Home',
  'Punt_Away',
  'Pass_Incompletion_Away',
  'Timeout_Away',
  'homeScore',
  'awayScore',
  'PassReception_Home_yards',
  'PassReception_Away_yards',
  'Penalty_Home',
  'Penalty_Away',
  'rushing_yards_home',
  'rushing_yards_away',
  'field_goals_good_home',
  'field_goals_good_away',
  'field_goals_missed_home',
  'field_goals_missed_away',
  'fumble_recovery_oponnent_home',
  'fumble_recovery_oponnent_away',
  'fumble_recovery_own_home',
  'fumble_recovery_own_away',
  'sack_home',
  'sack_away',
  'passing_TD_home',
  'passing_TD_away',
  'rushing_TD_home',
  'rushing_TD_away',
  'interception_home',
  'interception_away',
  'Interception_Return_TD_home',
  'Interception_Return_TD_away',
  'kick_off_return_TD_home',
  'kick_off_return_TD_away',
  'safety_home',
  'safety_away',
  'homeWinPercentage',
  'awayWinPercentage'])

In [12]:
def stats(GameID, Quarter, time):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
    response_drive = r.get(url)
    drive_data = response_drive.json()
    drive_df = json_normalize(drive_data['items'])
    data = []
    time = datetime.timedelta(seconds=time)
    clock = datetime.timedelta(seconds=0)
    Pass_Incompletion_Home = 0
    Punt_Home = 0
    Timeout_Home = 0
    Punt_Away = 0
    Pass_Incompletion_Away = 0
    Timeout_Away = 0
    homeScore = 0
    awayScore = 0
    PassReception_Home_yards = 0
    PassReception_Away_yards = 0
    Penalty_Home = 0
    Penalty_Away = 0
    rushing_yards_home = 0
    rushing_yards_away = 0
    field_goals_good_home = 0
    field_goals_good_away = 0
    field_goals_missed_home = 0
    field_goals_missed_away = 0
    fumble_recovery_oponnent_home = 0
    fumble_recovery_oponnent_away = 0
    fumble_recovery_own_home = 0
    fumble_recovery_own_away = 0
    passing_TD_home = 0
    passing_TD_away = 0
    rushing_TD_home = 0
    rushing_TD_away = 0
    interception_home = 0
    interception_away = 0
    Interception_Return_TD_home = 0
    Interception_Return_TD_away = 0
    kick_off_return_TD_home = 0
    kick_off_return_TD_away = 0
    safety_home = 0
    safety_away = 0
    sack_home = 0
    sack_away = 0
    homeWinPercentage = 0
    awayWinPercentage = 0
    for index, row in drive_df.iterrows():
        try:
            if (re.search(r"teams/(\d+)", row['team.$ref']).group(1) ==
                    Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
                for detail in range(len(row['plays.items'])):
                    duration_str = row['plays.items'][detail]['clock']['displayValue']
                    duration_arr = duration_str.split(":")
                    duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
                    clock = datetime.timedelta(seconds=900) - duration
                    if (row['plays.items'][detail]['period']['number'] == Quarter and clock <= time) or (row['plays.items'][detail]['period']['number'] < Quarter):
                        try:
                            string_url=row['plays.items'][detail]['probability']['$ref']
                            url_probability=string_url
                            response_probability = r.get(url_probability)
                            probability_data = response_probability.json()
                            homeWinPercentage = probability_data['homeWinPercentage']
                            awayWinPercentage = 1 - homeWinPercentage
                        except KeyError:
                            print('it doenst worked')                     
                        if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
                            if row['plays.items'][detail]['statYardage'] > 0:
                              Penalty_Away = Penalty_Away  + 1
                            elif row['plays.items'][detail]['statYardage'] < 0:
                              Penalty_Home = Penalty_Home + 1
                        if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Incompletion':
                            Pass_Incompletion_Home = Pass_Incompletion_Home + 1
                            PassReception_Home_yards = PassReception_Home_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Punt':
                            Punt_Home = Punt_Home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Timeout':
                            Timeout_Home = Timeout_Home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Reception':
                            PassReception_Home_yards = PassReception_Home_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rush':
                            rushing_yards_home = rushing_yards_home + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Sack':
                            sack_home = sack_home + 1
                            rushing_yards_home = row['plays.items'][detail]['statYardage'] + rushing_yards_home
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Passing Touchdown':
                            passing_TD_home = passing_TD_home + 1
                            PassReception_Home_yards = PassReception_Home_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rushing Touchdown':
                            rushing_TD_home = rushing_TD_home + 1
                            rushing_yards_home = rushing_yards_home + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Interception Return':
                            interception_home = interception_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Good':
                            field_goals_good_home = field_goals_good_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Missed':
                            field_goals_missed_home = field_goals_missed_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Interception Return Touchdown':
                            Interception_Return_TD_home = Interception_Return_TD_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Kickoff Return Touchdown':
                            kick_off_return_TD_home = kick_off_return_TD_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Own)':
                            fumble_recovery_own_home = fumble_recovery_own_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Opponent)':
                            fumble_recovery_oponnent_home = fumble_recovery_oponnent_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Safety':
                            safety_home = safety_home + 1
                        homeScore = row['plays.items'][detail]['homeScore']
            elif (re.search(r"teams/(\d+)", row['team.$ref']).group(1) !=
                  Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
                for detail in range(len(row['plays.items'])):
                    duration_str2 = row['plays.items'][detail]['clock']['displayValue']
                    duration_arr2 = duration_str2.split(":")
                    duration2 = datetime.timedelta(minutes=int(duration_arr2[0]), seconds=int(duration_arr2[1]))
                    clock = datetime.timedelta(seconds=900) - duration2
                    if (row['plays.items'][detail]['period']['number'] == Quarter and clock <= time) or (row['plays.items'][detail]['period']['number'] < Quarter):
                        try:
                            string_url=row['plays.items'][detail]['probability']['$ref']
                            url_probability=string_url
                            response_probability = r.get(url_probability)
                            probability_data = response_probability.json()
                            homeWinPercentage = probability_data['homeWinPercentage']
                            awayWinPercentage = 1 - homeWinPercentage
                        except KeyError:
                            print('it doenst worked')
                        if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
                            if row['plays.items'][detail]['statYardage'] > 0:
                              Penalty_Away = Penalty_Away  + 1
                            elif row['plays.items'][detail]['statYardage'] < 0:
                              Penalty_Home = Penalty_Home + 1
                        if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Incompletion':
                            Pass_Incompletion_Away = Pass_Incompletion_Away + 1
                            PassReception_Away_yards = PassReception_Away_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Punt':
                            Punt_Away = Punt_Away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Timeout':
                            Timeout_Away = Timeout_Away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Reception':
                            PassReception_Away_yards = PassReception_Away_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rush':
                            rushing_yards_away = rushing_yards_away + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Sack':
                            rushing_yards_away = row['plays.items'][detail]['statYardage'] + rushing_yards_away
                            sack_away = sack_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Passing Touchdown':
                            passing_TD_away = passing_TD_away + 1
                            PassReception_Away_yards = PassReception_Away_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rushing Touchdown':
                            rushing_TD_away = rushing_TD_away + 1
                            rushing_yards_away = rushing_yards_away + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Interception Return':
                            interception_away = interception_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Good':
                            field_goals_good_away = field_goals_good_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Missed':
                            field_goals_missed_away = field_goals_missed_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Interception Return Touchdown':
                            Interception_Return_TD_away = Interception_Return_TD_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Kickoff Return Touchdown':
                            kick_off_return_TD_away = kick_off_return_TD_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Own)':
                            fumble_recovery_own_away = fumble_recovery_own_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Opponent)':
                            fumble_recovery_oponnent_away = fumble_recovery_oponnent_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Safety':
                            safety_away = safety_away + 1
                        awayScore = row['plays.items'][detail]['awayScore']
        except TypeError:
            print('it doenst worked')
    data = [GameID, clock, Pass_Incompletion_Home,
            Punt_Home, Timeout_Home,
            Punt_Away, Pass_Incompletion_Away,
            Timeout_Away, homeScore,
            awayScore, PassReception_Home_yards,
            PassReception_Away_yards, Penalty_Home,
            Penalty_Away,
            rushing_yards_home,
            rushing_yards_away, field_goals_good_home,
            field_goals_good_away, field_goals_missed_home,
            field_goals_missed_away, fumble_recovery_oponnent_home,
            fumble_recovery_oponnent_away, fumble_recovery_own_home,
            fumble_recovery_own_away, sack_home,
            sack_away, passing_TD_home,
            passing_TD_away, rushing_TD_home,
            rushing_TD_away, interception_home,
            interception_away, Interception_Return_TD_home,
            Interception_Return_TD_away, kick_off_return_TD_home,
            kick_off_return_TD_away,
            safety_home, safety_away, homeWinPercentage, awayWinPercentage]
    stats_df.loc[len(stats_df)] = data




In [ ]:
GameID='401437957'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
homescore=0
awayScore=0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0
rush = 0
penalty_home = 0
penalty_away = 0
plays = 0
for index, row in drive_df.iterrows():
  if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
          for detail in range(len(row['plays.items'])):
            plays = plays + 1
            #print(row['plays.items'][detail]['alternativeText'])
            if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
              
              print(row['plays.items'][detail]['alternativeText'])
              #print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
              print(row['plays.items'][detail]['statYardage'])
              if row['plays.items'][detail]['statYardage'] > 0:
                penalty_away = penalty_away  + 1
              elif row['plays.items'][detail]['statYardage'] < 0:
                penalty_home = penalty_home + 1
                print(penalty_home)
  elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
          for detail in range(len(row['plays.items'])):
            plays = plays + 1
            #print(row['plays.items'][detail]['alternativeText'])
            if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
              print(row['plays.items'][detail]['alternativeText'])
              print(row['plays.items'][detail]['statYardage'])

              #print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
              if row['plays.items'][detail]['statYardage'] < 0:
                penalty_away = penalty_away  + 1
              elif row['plays.items'][detail]['statYardage'] > 0:
                penalty_home = penalty_home + 1
                
                
print(penalty_home)
print(penalty_away)


(1:47) (Shotgun) D.Webb scrambles right end pushed ob at PHI 46 for 8 yards (T.Edwards). PENALTY on PHI-T.Edwards, Unnecessary Roughness, 15 yards, enforced at PHI 46.
23
(12:05) J.Gillan punts 46 yards to PHI 35, Center-C.Kreiter. B.Covey ran ob at NYG 49 for 16 yards (L.Collins). PENALTY on NYG-L.Collins, Offensive Holding, 10 yards, enforced at NYG 49.
16
(10:07) (Shotgun) PENALTY on PHI-B.Graham, Neutral Zone Infraction, 5 yards, enforced at NYG 25 - No Play.
5
(7:28) (Shotgun) PENALTY on NYG-E.Neal, False Start, 5 yards, enforced at PHI 40 - No Play.
-5
(7:04) (Shotgun) D.Webb pass incomplete short right [H.Reddick]. PENALTY on NYG-D.Webb, Intentional Grounding, 16 yards, enforced at PHI 45.
-16
(2:00) (Shotgun) J.Hurts pass short right to A.Brown ran ob at NYG 36 for 11 yards. PENALTY on NYG-N.McCloud, Roughing the Passer, 15 yards, enforced at NYG 36.
26
(:19) (Shotgun) PENALTY on NYG-E.Neal, False Start, 5 yards, enforced at NYG 37 - No Play.
-5
(12:19) (Shotgun) J.Hurts pass s

In [ ]:
GameID='401437957'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
homescore = 0
awayScore = 0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0
rush = 0
penalty_home = 0
penalty_away = 0
plays = 0
for index, row in drive_df.iterrows():
  for detail in range(len(row['plays.items'])):
    id = row['plays.items'][detail]['id']
    try:
      string_url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/probabilities/{id}?lang=en&region=us'
      url_probability=string_url
      response_probability = r.get(url_probability)
      probability_data = response_probability.json()
      probability_df = json_normalize(probability_data)
      print(probability_data['homeWinPercentage'])
    except KeyError:
      print('it doenst worked')

print(penalty_home)
print(penalty_away)

0.9281
0.9317
0.9321
0.9355
0.9371
0.9487
0.9431
0.9533
0.9483
0.9438
0.938
0.9396
0.9387
0.9427
0.9391
0.9478
0.9439
0.942
0.9377
0.9452
0.9395
0.9353
0.9532
0.9533
0.956
0.952
0.9594
0.9524
0.9622
0.9615
0.9641
0.963
0.9552
0.9579
0.9453
0.9473
0.9492
0.952
0.9655
0.9654
0.9627
0.9618
0.9558
0.9571
0.9568
0.96
0.9623
0.9667
0.9622
0.962
0.958
0.9658
0.9652
0.9678
0.9692
0.9664
0.9627
0.9635
0.9632
0.9662
0.953
0.954
0.9619
0.9635
0.9665
0.9659
0.9677
0.9675
0.9654
0.9664
0.9689
0.9671
0.971
0.967
0.9668
0.9774
0.9807
0.9798
0.9741
0.9703
0.9711
0.9722
0.9679
0.9667
0.9757
0.9752
0.9756
0.9736
0.9753
0.9767
0.9763
0.977
0.9755
0.9759
0.9808
0.9856
0.9795
0.9839
0.9812
0.9862
0.9886
0.986
0.9846
0.9854
0.9728
0.9747
0.9696
0.9649
0.9726
0.9769
0.9778
0.9804
0.9777
0.9832
0.9806
0.9788
0.9762
0.9861
0.9854
0.9813
0.9801
0.9733
0.9758
0.9645
0.973
0.9756
0.978
0.9778
0.9843
0.9836
0.9844
0.9827
0.981
0.9765
0.9767
0.974
0.9776
0.981
0.9852
0.9874
0.9749
0.9828
0.9715
0.9591
0.9653
0.9655

In [ ]:
#for plays in Data_basic['GameID']:
  #stats(plays,4,900)
  

KeyboardInterrupt: ignored

In [ ]:
#stats_df.to_csv('/content/drive/MyDrive/quarter1_data.csv', index=False)

In [33]:
stats('401326322',4,820)

In [34]:
stats_df

,GameID,clock,Pass_Incompletion_Home,Punt_Home,Timeout_Home,Punt_Away,Pass_Incompletion_Away,Timeout_Away,homeScore,awayScore,...,interception_home,interception_away,Interception_Return_TD_home,Interception_Return_TD_away,kick_off_return_TD_home,kick_off_return_TD_away,safety_home,safety_away,homeWinPercentage,awayWinPercentage
0,401326322,0 days 00:14:58,4,3,0,1,3,0,7,7,...,0,0,0,0,0,0,0,0,0.6783,0.3217
1,401326322,0 days 00:14:58,6,3,0,2,12,2,21,16,...,2,0,0,0,0,0,0,0,0.7074,0.2926
2,401326322,0 days 00:14:58,4,1,0,1,3,0,7,7,...,0,0,0,0,0,0,0,0,0.6352,0.3648
3,401326322,0 days 00:14:58,16,5,1,3,18,5,31,29,...,2,1,0,0,0,0,0,0,0.9949,0.0051
4,401326322,0 days 00:14:58,11,5,0,3,18,4,28,26,...,2,1,0,0,0,0,0,0,0.6391,0.3609
5,401326322,0 days 00:14:58,11,5,0,3,18,5,28,26,...,2,1,0,0,0,0,0,0,0.5444,0.4556
6,401326322,0 days 00:14:58,11,5,0,3,18,5,28,29,...,2,1,0,0,0,0,0,0,0.4507,0.5493


In [ ]:

url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401326316/competitions/401326316/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
 



In [ ]:
import datetime

duration_str = "15:00"
duration_arr = duration_str.split(":")
duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))

print(duration)

In [ ]:
GameID='401326322'
for index, row in drive_df.iterrows():
  for detail in range(len(row['plays.items'])):
    #print(row['plays.items'][detail])

    duration_str = row['plays.items'][detail]['clock']['displayValue']
    duration_arr = duration_str.split(":")
    duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
    print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
    if (datetime.timedelta(seconds=900)-duration) <= (datetime.timedelta(seconds=60)):
      print(datetime.timedelta(seconds=900)-duration)

In [ ]:
print(duration-datetime.timedelta(seconds=110))

In [ ]:
15*60